In [1]:
# data libraries
import pandas as pd
import numpy as np

#plotting libraries
import plotly.express as px

#python libraries
import random
import os
import glob
import requests
import json
import time

#tqdm
from tqdm import tqdm

In [2]:
class CONFIG:
    hdb_property_fp = os.path.join("..","data", "hdb-property-information", "hdb-property-information.csv")
    hdb_resale_price_fps = glob.glob("../data/resale-flat-prices/*.csv")
    hdb_resale_price_fp = os.path.join("..","data","resale-flat-prices", 
                                        "resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")

## Loading in the property information data

In [3]:
hdb_property_df = pd.read_csv(CONFIG.hdb_property_fp)
hdb_property_df['address'] =hdb_property_df['blk_no'] + " " + hdb_property_df['street']
hdb_property_df = hdb_property_df.drop(["1room_sold", "2room_sold", 
                                        "3room_sold", "4room_sold", 
                                        "5room_sold", "exec_sold",
                                        "multigen_sold", "studio_apartment_sold",
                                        "1room_rental", "2room_rental",
                                        "3room_rental", "other_room_rental", "bldg_contract_town",
                                        "other_room_rental"], axis=1)
display(hdb_property_df.head())
display(hdb_property_df.info())

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,total_dwelling_units,address
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,142,1 BEACH RD
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,206,1 BEDOK STH AVE 1
2,1,CANTONMENT RD,2,2010,N,Y,N,N,N,N,0,1 CANTONMENT RD
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,102,1 CHAI CHEE RD
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,55,1 CHANGI VILLAGE RD


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12442 entries, 0 to 12441
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   blk_no                12442 non-null  object
 1   street                12442 non-null  object
 2   max_floor_lvl         12442 non-null  int64 
 3   year_completed        12442 non-null  int64 
 4   residential           12442 non-null  object
 5   commercial            12442 non-null  object
 6   market_hawker         12442 non-null  object
 7   miscellaneous         12442 non-null  object
 8   multistorey_carpark   12442 non-null  object
 9   precinct_pavilion     12442 non-null  object
 10  total_dwelling_units  12442 non-null  int64 
 11  address               12442 non-null  object
dtypes: int64(3), object(9)
memory usage: 1.1+ MB


None

## We want to get the geo-coordinate information in order to plot on mapbox 3d map
1. we will use a kaggle dataset to get as much coordinates as we can and get the remaining via webscapping
    - https://www.kaggle.com/denzilg/hdb-flat-prices-19902021-march?select=ALL+Prices+1990-2021+mar.csv

In [4]:
coordinates = pd.read_csv("../data/hdb_coordinates.csv")
coordinates['address'] = coordinates['address']
coordinates.head()

,address,latitude,longitude
0,308 ANG MO KIO AVE 1,1.365485,103.844025
1,216 ANG MO KIO AVE 1,1.366272,103.841465
2,211 ANG MO KIO AVE 3,1.369226,103.841652
3,254 ANG MO KIO AVE 4,1.370276,103.834699
4,231 ANG MO KIO AVE 3,1.368640,103.837490


## Merging hdb_property

In [5]:
hdb_property_w_coordinate_df = hdb_property_df.merge(coordinates, how='left', on='address')
hdb_property_w_coordinate_df.head()

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,total_dwelling_units,address,latitude,longitude
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,142,1 BEACH RD,1.303489,103.864529
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,206,1 BEDOK STH AVE 1,1.320836,103.933675
2,1,CANTONMENT RD,2,2010,N,Y,N,N,N,N,0,1 CANTONMENT RD,NaN,NaN
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,102,1 CHAI CHEE RD,1.328015,103.922555
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,55,1 CHANGI VILLAGE RD,1.388528,103.987631


## We are still missing 3232 coordinates

In [6]:
hdb_property_w_coordinate_df["latitude"].isna().sum()

3232

In [7]:
hdb_property_without_coordinate_df = hdb_property_w_coordinate_df[hdb_property_w_coordinate_df["latitude"].isna()]
hdb_property_w_coordinate_df = hdb_property_w_coordinate_df[~hdb_property_w_coordinate_df["latitude"].isna()]

## Lets scrap the coordinates from the web

In [165]:
def get_coordinate_from_geocode(address):
    """
    This function will take in address str and send http request to geocode
    """
    try:
        req = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
        result_dict = json.loads(req.text)['results'][0]
        latitude = result_dict['LATITUDE']
        longitude =  result_dict['LONGITUDE']
        return latitude, longitude
    except Exception as e:
        print(e)
        return None, None

def get_coordinates(address_list):
    latitude_list=[]
    longitude_list=[]
    
    for address in tqdm(address_list):
        lat, long = get_coordinate_from_geocode(address)
        latitude_list.append(lat)
        longitude_list.append(long)
        time.sleep(0.5)
        
    return latitude_list, longitude_list

In [166]:
latitude_list, longitude_list = get_coordinates(hdb_property_without_coordinate_df['address'].to_list())

 19%|███████▍                                | 599/3232 [06:24<28:06,  1.56it/s]

list index out of range


 29%|███████████▌                            | 936/3232 [09:59<22:07,  1.73it/s]

list index out of range


 30%|███████████▊                            | 957/3232 [10:13<31:46,  1.19it/s]

...

 47%|██████████████████▏                    | 1507/3232 [17:25<18:45,  1.53it/s]

list index out of range


 61%|███████████████████████▋               | 1958/3232 [22:29<13:10,  1.61it/s]

list index out of range


100%|███████████████████████████████████████| 3232/3232 [36:23<00:00,  1.48it/s]


In [167]:
hdb_property_without_coordinate_df['latitude'] = latitude_list
hdb_property_without_coordinate_df["longitude"] = longitude_list

## There are 10 entries can did not return anything so we will manually add those in

In [8]:
# hdb_property_without_coordinate_df.to_csv("../data/hdb_property_coordinate.csv")
hdb_property_without_coordinate_df = pd.read_csv("../data/hdb_property_coordinate_reviewed.csv", index_col=0)

In [9]:
hdb_property_w_coordinate_df = pd.concat([hdb_property_w_coordinate_df,hdb_property_without_coordinate_df])

In [10]:
hdb_property_w_coordinate_df.head()

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,total_dwelling_units,address,latitude,longitude
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,142,1 BEACH RD,1.303489,103.864529
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,206,1 BEDOK STH AVE 1,1.320836,103.933675
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,102,1 CHAI CHEE RD,1.328015,103.922555
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,55,1 CHANGI VILLAGE RD,1.388528,103.987631
5,1,DELTA AVE,25,1982,Y,N,N,N,N,N,96,1 DELTA AVE,1.292047,103.826927


## Loading in resale price data

In [13]:
hdb_resale_prices = []
for fp in CONFIG.hdb_resale_price_fps:
    hdb_resale_prices.append(pd.read_csv(fp))
hdb_resale_price_df = pd.concat(hdb_resale_prices).reset_index(drop=True)
display(hdb_resale_price_df.head())
display(hdb_resale_price_df.describe())

# hdb_resale_price_df = pd.read_csv(CONFIG.hdb_resale_price_fp)
hdb_resale_price_df['address'] = hdb_resale_price_df['block'] + " " + hdb_resale_price_df["street_name"]
display(hdb_resale_price_df.head())
hdb_resale_price_df.info()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0


,floor_area_sqm,lease_commence_date,resale_price
count,203109.000000,203109.000000,2.031090e+05
mean,97.278123,1992.845940,4.548632e+05
std,24.385624,12.435018,1.462102e+05
min,31.000000,1966.000000,1.400000e+05
25%,76.000000,1984.000000,3.500000e+05
50%,95.000000,1992.000000,4.280000e+05
75%,112.000000,2002.000000,5.280000e+05
max,280.000000,2019.000000,1.295000e+06


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,541 ANG MO KIO AVE 10
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,163 ANG MO KIO AVE 4
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,446 ANG MO KIO AVE 10
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,557 ANG MO KIO AVE 10


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203109 entries, 0 to 203108
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                203109 non-null  object 
 1   town                 203109 non-null  object 
 2   flat_type            203109 non-null  object 
 3   block                203109 non-null  object 
 4   street_name          203109 non-null  object 
 5   storey_range         203109 non-null  object 
 6   floor_area_sqm       203109 non-null  float64
 7   flat_model           203109 non-null  object 
 8   lease_commence_date  203109 non-null  int64  
 9   remaining_lease      150906 non-null  object 
 10  resale_price         203109 non-null  float64
 11  address              203109 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 18.6+ MB


## imputing Remaining_lease column

In [15]:
sub_df = hdb_resale_price_df.loc[hdb_resale_price_df["remaining_lease"].isna()].copy()
hdb_resale_price_df.loc[hdb_resale_price_df["remaining_lease"].isna(),"remaining_lease"] = (99-(pd.to_datetime(sub_df["month"]).dt.year-sub_df['lease_commence_date'])).astype(str) + " years"

## Joining the df together before preprocessing the data

In [17]:
raw_df = hdb_resale_price_df.merge(hdb_property_w_coordinate_df, how='left', on='address')

In [18]:
print(f"missing values due to joining {raw_df['latitude'].isna().sum()}")

missing values due to joining 4


In [19]:
#drop missing value as we dont even have the property information
raw_df = raw_df.dropna()

In [20]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203105 entries, 0 to 203108
Data columns (total 25 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   month                 203105 non-null  object 
 1   town                  203105 non-null  object 
 2   flat_type             203105 non-null  object 
 3   block                 203105 non-null  object 
 4   street_name           203105 non-null  object 
 5   storey_range          203105 non-null  object 
 6   floor_area_sqm        203105 non-null  float64
 7   flat_model            203105 non-null  object 
 8   lease_commence_date   203105 non-null  int64  
 9   remaining_lease       203105 non-null  object 
 10  resale_price          203105 non-null  float64
 11  address               203105 non-null  object 
 12  blk_no                203105 non-null  object 
 13  street                203105 non-null  object 
 14  max_floor_lvl         203105 non-null  float64
 15  

In [21]:
raw_df.isna().sum()

month                   0
town                    0
flat_type               0
block                   0
street_name             0
storey_range            0
floor_area_sqm          0
flat_model              0
lease_commence_date     0
remaining_lease         0
resale_price            0
address                 0
blk_no                  0
street                  0
max_floor_lvl           0
year_completed          0
residential             0
commercial              0
market_hawker           0
miscellaneous           0
multistorey_carpark     0
precinct_pavilion       0
total_dwelling_units    0
latitude                0
longitude               0
dtype: int64

In [22]:
raw_df.to_csv("../data/raw_df.csv", index=False)